In [ ]:
import pyodbc
import os
import json
import logging
from datetime import datetime

class SQLDatabase:
    def __init__(self, server_name, database_name, user_name, password, driver=driver):
        self.server_name = server_name
        self.database_name = database_name
        self.user_name = user_name
        self.password = password
        self.driver = driver
        self.connection = self.connect_to_database()

    def connect_to_database(self):
        connection_string = f"DRIVER={{{self.driver}}};SERVER={self.server_name};DATABASE={self.database_name};UID={self.user_name};PWD={self.password};"
        return pyodbc.connect(connection_string)

    def execute_query(self, query, isResult = False):
        cursor = self.connection.cursor()
        cursor.execute(query)
        if isResult:
            result = int(cursor.fetchone()[0])
        else:
            result = None
        self.connection.commit()
        cursor.close()
        return result

    def close_connection(self):
        self.connection.close()

class ViewManager:
    def __init__(self, sql_db, data_source):
        self.sql_db = sql_db
        self.data_source = data_source

    def drop_view(self, schema, view_name):
        query = f"DROP VIEW IF EXISTS [{schema}].[{view_name}]"
        self.sql_db.execute_query(query)
    
    def create_schema_if_not_exists(self, schema):
        # NOTE: Under Dev & Review
        query = f"SELECT count(*) FROM sys.schemas WHERE name = N'{schema}';"        
        isSchema = self.sql_db.execute_query(query, isResult = True)
        if isSchema == 0:
            logger.info(f'CREATING SCHEMA {schema}')
            query = f"CREATE SCHEMA [{schema}] AUTHORIZATION [dbo]"
            self.sql_db.execute_query(query)
                
    def create_view(self, schema, view_name, file_path):
        query = f"""CREATE VIEW [{schema}].[{view_name}]
                AS SELECT *
                FROM  
                    OPENROWSET(
                        BULK '{file_path}',
                        DATA_SOURCE = '{self.data_source}',
                        FORMAT='DELTA'
                    ) nyc"""
        self.sql_db.execute_query(query)

    def add_view_to_sql_db(self, schema, view_name, file_path, overwrite=True):
        if overwrite:
            logging.info('Overwrite Mode Active - Dropping SQL DB Table')
            self.drop_view(schema, view_name)
        else:
            logging.info('Overwrite Mode Inactive')
        self.create_view(schema, view_name, file_path)

    def create_and_populate_views(self, schema, schema_path, view_names, overwrite=False):
        self.create_schema_if_not_exists(schema)
        for view_name in view_names:
            logging.info(f'Creating View - {view_name}')
            view_path = f"{schema_path}/{view_name}"
            try:
                self.add_view_to_sql_db(schema, view_name, view_path, overwrite)
            except Exception as error:
                logging.error(f'An error occurred while creating the SQL DB Table - {error}')